First Cell involved importing the two datasets, merging them into one dataframe, shuffling the data
and partitioning the data into training and test data with a 0.20 test proportion. Both the training and test data are further partitioned into X and y components, denoting each entries (tweets) and labels (0 or 1) respectively.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

gen = pd.read_csv('genuine1.csv')
poli = pd.read_csv('Political_Cleaned1.csv')
df0 = pd.concat([gen,poli],axis=0,sort=True)
df = df0.drop(['1'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.20)


Next we apply feature extraction to the X data. To do this we use a TfidfVectorizer which extracts a word dictionary from the data and creates a sparse matrix where for each row (data entry) we have the column indicating the prevelance of a specific word. The term frequency–inverse document frequency makes it so that rare words are weighted higher than frequent ones. Our new features allow us to work numerically since X has been tranformed from rows of tweets into rows of floating point numbers.

In [2]:

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train.values.astype('U'))
vectorTrain = vectorizer.transform(X_train.values.astype('U'))
vectorTest = vectorizer.transform(X_test.values.astype('U'))

We apply a Logistic Regression to our numeric vector. The lbfgs solver works well for sparse matrices and the low overhead for Logistic Regression allows us to use our full dataset without dimensionality reduction.

In [24]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
lgr = LogisticRegression(solver = 'lbfgs',max_iter = 500)
clf = GridSearchCV(lgr, parameters, cv=5)
clf.fit(vectorTrain,y_train)

C:\tools\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\tools\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\tools\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\tools\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\tools\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\tools\Anaconda3\l

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
best_pred = clf.predict(vectorTrain)
from sklearn import metrics
trainAcc = metrics.accuracy_score(y_train,best_pred)
print("trainAcc",trainAcc)

best_predTest = clf.predict(vectorTest)
testAcc = metrics.accuracy_score(y_test,best_predTest)
print("testAcc",testAcc)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
trainAcc 0.9630630403124658
testAcc 0.9028619528619528


In [31]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
lgr.fit(vectorTrain,y_train)
trainP= lgr.predict(vectorTrain)
from sklearn import metrics
trainA = metrics.accuracy_score(y_train,trainPredictions)
print("trainAcc",trainA)
testP = lgr.predict(vectorTest)
testA = metrics.accuracy_score(y_test,testPredictions)
print("testAcc",testA)
#testVector = vectorizer.transform(X_test.values.astype('U'))
#p = lgr.predict(testVector)
#from sklearn import metrics
#cm = metrics.accuracy_score(y_test, p)

trainAcc 0.9238280427956935
testAcc 0.9008754208754208


In [4]:
#from sklearn.linear_model import LogisticRegressionCV
#clf = LogisticRegressionCV(cv=5, random_state=0,multi_class='ovr',max_iter = 200).fit(vector, y_train)
#p2 = clf.predict(testVector)
#clf.score(testVector,y_test)